In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
import pandas as pd

# Cargar los datos de entrenamiento y prueba
test_data = pd.read_csv('/content/test_hogares_final.csv')
train_data = pd.read_csv('/content/train_hogares_final.csv')

# Seleccionar las variables predictoras y la variable objetivo
X_train = train_data[['Clase', 'Dominio', 'P5000', 'P5010', 'P5090', 'P5100',
                         'P5130', 'P5140', 'Nper', 'Npersug', 'Li', 'Lp', 'Fex_c',
                         'Depto', 'Fex_dpto', 'Vivienda_Propia_Pagada', 'Vivienda_Pagando',
                         'Vivienda_Arriendo', 'Vivienda_Usufructo', 'Vivienda_Posesion_Sin_Titulo',
                         'Vivienda_Otra', 'Indicador_Arriendo', 'Indicador_Hacinamiento_Critico',
                         'Num_Personas_Trabajando', 'Proporción_Ocupados', 'Promedio_Edad_Ocupados',
                         'Num_Personas_Estudiando', 'Proporción_Estudiando', 'Num_Dependientes',
                         'Proporción_Dependientes', 'Num_Mujeres', 'Proporción_Mujeres',
                         'Nivel_Educativo_Maximo', 'Max_Ninguno', 'Max_Preescolar',
                         'Max_Basica_Primaria', 'Max_Basica_Secundaria', 'Max_Media', 'Max_Superior',
                         'Num_Personas_Media_Superior', 'Indicador_Hogar_Jefe_Femenino']]

y_train = train_data['Pobre']  # La variable objetivo en el conjunto de entrenamiento

X_test = test_data[['Clase', 'Dominio', 'P5000', 'P5010', 'P5090', 'P5100',
                       'P5130', 'P5140', 'Nper', 'Npersug', 'Li', 'Lp', 'Fex_c',
                       'Depto', 'Fex_dpto', 'Vivienda_Propia_Pagada', 'Vivienda_Pagando',
                       'Vivienda_Arriendo', 'Vivienda_Usufructo', 'Vivienda_Posesion_Sin_Titulo',
                       'Vivienda_Otra', 'Indicador_Arriendo', 'Indicador_Hacinamiento_Critico',
                       'Num_Personas_Trabajando', 'Proporción_Ocupados', 'Promedio_Edad_Ocupados',
                       'Num_Personas_Estudiando', 'Proporción_Estudiando', 'Num_Dependientes',
                       'Proporción_Dependientes', 'Num_Mujeres', 'Proporción_Mujeres',
                       'Nivel_Educativo_Maximo', 'Max_Ninguno', 'Max_Preescolar',
                       'Max_Basica_Primaria', 'Max_Basica_Secundaria', 'Max_Media', 'Max_Superior',
                       'Num_Personas_Media_Superior', 'Indicador_Hogar_Jefe_Femenino']]

# Excluir las columnas con muchos valores faltantes
columns_to_exclude = ['P5100', 'P5140', 'P5130', 'Promedio_Edad_Ocupados']

# Crear nuevas versiones de los conjuntos de datos excluyendo estas columnas
X_train_selected = X_train.drop(columns=columns_to_exclude)
X_test_selected = X_test.drop(columns=columns_to_exclude)

# Convertir las variables categóricas en dummies (variables binarias)
X_train_selected = pd.get_dummies(X_train_selected, columns=['Clase', 'Dominio', 'Depto'])
X_test_selected = pd.get_dummies(X_test_selected, columns=['Clase', 'Dominio', 'Depto'])

# Alinear las columnas entre los conjuntos de entrenamiento y prueba
X_train_selected, X_test_selected = X_train_selected.align(X_test_selected, join='left', axis=1, fill_value=0)

# Imputar valores faltantes
imputer = SimpleImputer(strategy='mean')
X_train_selected = imputer.fit_transform(X_train_selected)
X_test_selected = imputer.transform(X_test_selected)

# Crear el modelo CART
cart_model = DecisionTreeClassifier()

# Entrenar el modelo con los datos de entrenamiento
cart_model.fit(X_train_selected, y_train)

# Hacer predicciones con el conjunto de prueba
y_pred_test_cart = cart_model.predict(X_test_selected)

# Crear el DataFrame
predicciones = pd.DataFrame({
    'id': test_data['id'],
    'Pobre_Pred': y_pred_test_cart
})

# Guardar el DataFrame
predicciones.to_csv('predicciones_kaggle_CART.csv', index=False)





Archivo 'predicciones_kaggle_CART.csv' generado con éxito.


In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# rango de hiperparámetros para probar
param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'criterion': ['gini', 'entropy']
}

# modelo CART
cart_model = DecisionTreeClassifier()

#  GridSearch para probar los hiperparámetros
grid_search = GridSearchCV(cart_model, param_grid, cv=5, scoring='accuracy')

# Entrenar el modelo con la búsqueda de los mejores hiperparámetros
grid_search.fit(X_train_selected, y_train)

# Ver los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", grid_search.best_params_)

# Usar el mejor modelo encontrado para hacer predicciones
best_cart_model = grid_search.best_estimator_
y_pred_test_cart = best_cart_model.predict(X_test_selected)

# Crear el DataFrame con las predicciones y los IDs
predicciones = pd.DataFrame({
    'id': test_data['id'],
    'Pobre_Pred': y_pred_test_cart
})

# Guardar el DataFrame como un archivo CSV para subir a Kaggle
predicciones.to_csv('predicciones_kaggle_CART_mejorado.csv', index=False)

print("Archivo 'predicciones_kaggle_CART_mejorado.csv' generado con éxito.")


Mejores hiperparámetros: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 10, 'min_samples_split': 2}
Archivo 'predicciones_kaggle_CART_mejorado.csv' generado con éxito.


In [8]:
from sklearn.metrics import classification_report, accuracy_score

y_train_pred_cart = best_cart_model.predict(X_train_selected)

# Evaluar el rendimiento del modelo en el conjunto de entrenamiento
print("Reporte de clasificación en el conjunto de entrenamiento:")
print(classification_report(y_train, y_train_pred_cart))

# Calcular y mostrar la exactitud (accuracy) en el conjunto de entrenamiento
accuracy_train_cart = accuracy_score(y_train, y_train_pred_cart)
print(f"Exactitud en el conjunto de entrenamiento: {accuracy_train_cart}")

# Calcular las métricas detalladas (precisión, recall, f1-score) y generar el reporte
reporte_clasificacion_cart = classification_report(y_train, y_train_pred_cart, output_dict=True)

# Crear un DataFrame para mostrar las métricas en forma tabular
df_reporte_clasificacion_cart = pd.DataFrame(reporte_clasificacion_cart).transpose()


Reporte de clasificación en el conjunto de entrenamiento:
              precision    recall  f1-score   support

           0       0.88      0.95      0.91    131936
           1       0.71      0.46      0.56     33024

    accuracy                           0.85    164960
   macro avg       0.79      0.71      0.74    164960
weighted avg       0.84      0.85      0.84    164960

Exactitud en el conjunto de entrenamiento: 0.8542555771096023
